# Prêt à dépenser : Construire un modèle de scoring


## Contexte

"Prêt à dépenser" (Home Credit) est une société financière qui propose des crédits à la consommation pour des personnes ayant peu ou pas d'historique de prêt.
Pour accorder un crédit à la consommation, l'entreprise calcule la probabilité qu'un client le rembourse, ou non. Elle souhaite donc développer un algorithme de scoring pour aider à décider si un prêt peut être accordé à un client.

Les chargés de relation client seront les utilisateurs du modèle de scoring. Puisqu'ils s'adressent aux clients, ils ont besoin que votre modèle soit facilement interprétable. Les chargés de relation souhaitent, en plus, disposer d'une mesure de l'importance des variables qui ont poussé le modèle à donner cette probabilité à un client.


## Chargement des modules du projet

Afin de simplifier le Notebook, le code métier du projet est placé dans le répertoire [src/](../src/).


In [1]:
# Import project modules from source directory

# Hide warnings
import warnings
warnings.simplefilter(action='ignore', category=[UserWarning])

# System modules
import os
import sys

# Append source directory to system path
src_path = os.path.abspath(os.path.join("../src"))
if src_path not in sys.path:
    sys.path.append(src_path)

# Helper functions
import data.helpers as data_helpers
import features.helpers as feat_helpers
import visualization.helpers as vis_helpers
import models.helpers as models_helpers



Nous allons utiliser le langage [Python](https://www.python.org/about/gettingstarted/), et présenter ici le code, les résultats et l'analyse sous forme de [Notebook JupyterLab](https://jupyterlab.readthedocs.io/en/stable/getting_started/overview.html).

Nous allons aussi utiliser les bibliothèques usuelles d'exploration et analyse de données, afin d'améliorer la simplicité et la performance de notre code :
  * [NumPy](https://numpy.org/doc/stable/user/quickstart.html) et [Pandas](https://pandas.pydata.org/docs/user_guide/index.html) : effectuer des calculs scientifiques (statistiques, algèbre, ...) et manipuler des séries et tableaux de données volumineuses et complexes
  * [scikit-learn](https://scikit-learn.org/stable/getting_started.html) et [XGBoost](https://xgboost.readthedocs.io/en/latest/get_started.html) : pour effectuer des analyses prédictives 
  * [Matplotlib](https://matplotlib.org/stable/tutorials/introductory/usage.html), [Pyplot](https://matplotlib.org/stable/tutorials/introductory/pyplot.html), [Seaborn](https://seaborn.pydata.org/tutorial/function_overview.html) et [Plotly](https://plotly.com/python/getting-started/) : générer des graphiques lisibles, intéractifs et pertinents


In [2]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd

# sklearn preprocessing for dealing with categorical variables
from sklearn.model_selection import train_test_split

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Accelerate the development cycle
SAMPLE_FRAC: float = 1

# Prevent excessive memory usage used by plotly
DRAW_PLOTS: bool = False


ENCODE: bool = True
ENGINEER: bool = False
SIMPLIFY:bool = False
SCALE:bool = False
IMPUTE:bool = True

## Chargement des données

Les données mises à disposition sont issues de [Home Credit](https://www.homecredit.net/) et plus précisément de la compétition hébergée sur Kaggle [Home Credit Default Risk - Can you predict how capable each applicant is of repaying a loan?](https://www.kaggle.com/c/home-credit-default-risk)

Les données sont fournies sous la forme de plusieurs fichiers CSV pouvant être liés entre eux de la manière suivante :

![Home Credit data relations](https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png)

_source : [Introduction: Home Credit Default Risk Competition](https://www.kaggle.com/willkoehrsen/start-here-a-gentle-introduction) by [Will Koehrsen](https://www.kaggle.com/willkoehrsen)_


In [3]:
# Download and extract the raw data
data_helpers.download_extract_zip(
    zip_file_url="https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/Parcours_data_scientist/Projet+-+Impl%C3%A9menter+un+mod%C3%A8le+de+scoring/Projet+Mise+en+prod+-+home-credit-default-risk.zip",
    files_names=(
        "application_test.csv",
        "application_train.csv",
        "bureau_balance.csv",
        "bureau.csv",
        "credit_card_balance.csv",
        "installments_payments.csv",
        "POS_CASH_balance.csv",
        "previous_application.csv",
    ),
    target_path="../data/raw/",
)


Nous allons charger toutes les données du fichier `application_train.csv` afin de travailler préparer les données du jeu d'entraînement et de test (variable `TARGET` vaut `O` : le client n'a pas fait défaut ou `1` : le client a fait défaut). Nous les séparerons les jeux de données au moment de l'entraînement et évaluation de nos modèles.

Les fichiers contiennent un grand nombre de variables booléennes et catégorielles que nous pouvons déjà typer comme telles.


In [4]:
# Read column names
application_train_column_names = pd.read_csv(
    "../data/raw/application_train.csv", nrows=0
).columns.values

# TARGET variable must be present in the Train datase
if "TARGET" not in application_train_column_names:
    raise ValueError(
        "TARGET column not found in application_train.csv. Please check that the file is not corrupted."
    )

# SK_ID_CURR variable must be present in the Train datase
if "SK_ID_CURR" not in application_train_column_names:
    raise ValueError(
        "SK_ID_CURR column not found in application_train_column_names.csv. Please check that the file is not corrupted."
    )

# Set column types according to fields description (../data/raw/HomeCredit_columns_description.csv)
# Categorical variables
column_types = {
    col: "category"
    for col in application_train_column_names
    if col.startswith(("NAME_",))
    or col.endswith(("_TYPE"))
    or col
    in [
        "CODE_GENDER",
        "WEEKDAY_APPR_PROCESS_START",
        "FONDKAPREMONT_MODE",
        "HOUSETYPE_MODE",
        "WALLSMATERIAL_MODE",
        "EMERGENCYSTATE_MODE",
    ]
}
# Boolean variables
column_types |= {
    col: bool
    for col in application_train_column_names
    if col.startswith(("FLAG_", "REG_", "LIVE_"))
}

# Load application data
app_df = pd.read_csv(
    "../data/raw/application_train.csv",
    dtype=column_types,
    true_values=["Y", "Yes", "1"],
    false_values=["N", "No", "0"],
    na_values=["XNA"], # bad values
)

# Sample to speed up development
if  float == type(SAMPLE_FRAC) and 0 < SAMPLE_FRAC < 1:
    app_df = app_df.sample(frac=SAMPLE_FRAC)

# Let's display basic statistical info about the data
app_df.describe(include="all")


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,307511.000000,307511.000000,307511,307507,307511,307511,307511.000000,3.075110e+05,3.075110e+05,307499.000000,...,307511,307511,307511,307511,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000
unique,NaN,NaN,2,2,2,2,NaN,NaN,NaN,NaN,...,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,Cash loans,F,False,True,NaN,NaN,NaN,NaN,...,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,278232,202448,202924,213312,NaN,NaN,NaN,NaN,...,305011,307328,307355,307408,NaN,NaN,NaN,NaN,NaN,NaN
mean,278180.518577,0.080729,NaN,NaN,NaN,NaN,0.417052,1.687979e+05,5.990260e+05,27108.573909,...,NaN,NaN,NaN,NaN,0.006402,0.007000,0.034362,0.267395,0.265474,1.899974
std,102790.175348,0.272419,NaN,NaN,NaN,NaN,0.722121,2.371231e+05,4.024908e+05,14493.737315,...,NaN,NaN,NaN,NaN,0.083849,0.110757,0.204685,0.916002,0.794056,1.869295
min,100002.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,2.565000e+04,4.500000e+04,1615.500000,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,189145.500000,0.000000,NaN,NaN,NaN,NaN,0.000000,1.125000e+05,2.700000e+05,16524.000000,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,278202.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,1.471500e+05,5.135310e+05,24903.000000,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,367142.500000,0.000000,NaN,NaN,NaN,NaN,1.000000,2.025000e+05,8.086500e+05,34596.000000,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000


In [5]:
app_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: bool(34), category(14), float64(65), int64(9)
memory usage: 187.7 MB


Le jeu de données contient 122 variables, dont la variable cible que nous devons estimer : `TARGET`. Parmis ces variables, nous avons :
- 34 variables booléennes
- 14 variables catégorielles
- 74 variables numériques

## Analyse exploratoire

Nous allons analyser la distribution de quelques variables.

### Variable cible

Voyons spécifiquement la distribution de la variable `TARGET` qui est celle que nous devrons estimer par la suite.
Les valeurs nulles représentent notre jeu d'entrainement.
Nous pouvons oberver que nous avons à faire à un problème de __classification binaire déséquilibré__ (il y a deux valeurs possibles, mais les deux valeurs ne sont pas également représentées).
Ceci va influencer la manière dont nous allons construire et entraîner notre modèle.


In [6]:
# Let's plot the distribution of the TARGET variable
if DRAW_PLOTS:
    fig = px.bar(
        app_df["TARGET"].replace({
            "0": "TARGET=0 : payments OK", 
            "1": "TARGET=1 : payment difficulties", 
        }).value_counts(),
        title="Distribution of TARGET variable",
        width=800,
        height=400,
    ).update_xaxes(
        title="TARGET",
    ).update_yaxes(title="Count")
    fig.show()


### Valeurs vides

Nous voyons que toutes les variables ont moins de 30% de valeurs vides, et près de la moitié a moins de 1% de valeurs vides. Le jeu de données est donc relativement bien rempli, ce qui ne devrait pas poser de problème pour la suite.


In [7]:
# Let's display variables with missing values ratio
if DRAW_PLOTS:
    vis_helpers.plot_empty_values(app_df)


### Valeurs impossibles

Quelques valeurs présentes dans les données semblent impossibles. Nous allons supprimer ces "outliers".


In [8]:
# Define data constraints
data_constraints = {
    "DAYS_EMPLOYED": {"min": -35000, "max": 0,}, # max 100 years, only negative values
}

if DRAW_PLOTS:
    # Let's display box plots for variables with outliers
    vis_helpers.plot_boxes(app_df, plot_columns=data_constraints.keys(), categorical_column="TARGET")

# Remove values that are outside possible range
app_df = feat_helpers.drop_impossible_values(
    app_df, constraints=data_constraints,
)


### Variables quantitatives

Nous allons simplement afficher la distribution de quelques variables numériques. Nous voyons déjà que selon la valeur de TARGET, la distribution (moyenne) des variables peut être sensiblement différente.


In [9]:
# Draw the BoxPlots of some numeric columns, split per Target
if DRAW_PLOTS:
    vis_helpers.plot_boxes(app_df,
        plot_columns=[
            "AMT_INCOME_TOTAL",
            "AMT_CREDIT",
            "AMT_ANNUITY",
            "AMT_GOODS_PRICE",
            "DAYS_BIRTH",
            "DAYS_EMPLOYED",
            "OWN_CAR_AGE",
            "REGION_RATING_CLIENT",
            "REGION_RATING_CLIENT_W_CITY",
            "EXT_SOURCE_1",
            "EXT_SOURCE_2",
            "EXT_SOURCE_3",
            "DAYS_LAST_PHONE_CHANGE",
            "AMT_REQ_CREDIT_BUREAU_YEAR",
        ],
        categorical_column="TARGET",
    )


### Variables qualitatives

De la même manière, nous allons simplement afficher la distribution de quelques variables catégorielles. Nous voyons déjà que selon la valeur de TARGET, la distribution (répartition entre classes) des variables peut être sensiblement différente (`TARGET=0` pour 77,6% des `NAME_CONTRACT_TYPE="Cash loans"`, tandis que `TARGET=0` pour 93,1% des `NAME_CONTRACT_TYPE="Revolving loans"`).
Certaines variables ont une répartition très inégale entre classes (`FLAG_MOBIL` vaut systématiquement `True` et jamais `False`).


In [10]:
# Draw the Bar charts of some categorical columns, split per Target
if DRAW_PLOTS:
    vis_helpers.plot_categories_bars(app_df,
        plot_columns=[
            "NAME_CONTRACT_TYPE",
            "CODE_GENDER",
            "FLAG_OWN_CAR",
            "FLAG_OWN_REALTY",
            "NAME_INCOME_TYPE",
            "NAME_EDUCATION_TYPE",
            "NAME_FAMILY_STATUS",
            "NAME_HOUSING_TYPE",
            "OCCUPATION_TYPE",
            "FLAG_MOBIL",
        ],
        categorical_column="TARGET",
    )


### Encodage des catégories

Lorsque les données qualitatives ne sont pas ordinales (on ne peu pas les classer selon un certain ordre), l'encodage "One Hot Encoding" sera plus performant que le "Label Encoding".


In [11]:
# Encode categorical variables with One Hot Encoding
if ENCODE:
    app_df = pd.get_dummies(app_df, dtype=bool)

    app_df.describe(include="bool")


Nous avons ici créé 134 nouvelles variables booléennes qui correspondent aux différentes classes de chacune des 14 anciennes variables catégorielles qui ont été encodées et supprimées.


## Feature Engineering

Nous allons tenter d'enrichir nos données en intégrant des variables qui sont des compositions non linéaires des variables existantes.


### Données métier

Afin d'apporter plus de sens aux données que nous allons fournir à nos modèles, nous pouvons faire appel aux experts métier qui peuvent nous indiquer des informations qui sont réputées importantes afin de prédire si un client risque d'avoir des problèmes de remboursement ou non.

Les informations métier pertinentes sont :
- Montant emprunté / Prix du bien acheté : `AMT_CREDIT / AMT_GOODS_PRICE`
- Montant des annuités / Montant emprunté : `AMT_ANNUITY / AMT_CREDIT`
- Montant des annuités / Revenu annuel : `AMT_ANNUITY / AMT_INCOME_TOTAL`
- Ancienneté au travail / Age : `DAYS_EMPLOYED / DAYS_BIRTH`


In [12]:
# Create the new features
if ENGINEER:
    app_df["CREDIT_PRICE_RATIO"] = app_df["AMT_CREDIT"] / app_df["AMT_GOODS_PRICE"]
    app_df["ANNUITY_CREDIT_RATIO"] = app_df["AMT_ANNUITY"] / app_df["AMT_CREDIT"]
    app_df["ANNUITY_INCOME_RATIO"] = app_df["AMT_ANNUITY"] / app_df["AMT_INCOME_TOTAL"]
    app_df["EMPLOYED_BIRTH_RATIO"] = app_df["DAYS_EMPLOYED"] / app_df["DAYS_BIRTH"]

    # Draw the BoxPlots for these features
    if DRAW_PLOTS:
        vis_helpers.plot_boxes(app_df,
            plot_columns=[
                "CREDIT_PRICE_RATIO",
                "ANNUITY_CREDIT_RATIO",
                "ANNUITY_INCOME_RATIO",
                "EMPLOYED_BIRTH_RATIO",
            ],
            categorical_column="TARGET",
        )


### Composition polynomiales de variables existantes

Les variables `EXT_SOURCE_{1-3}` n'ont a priori pas de sens concret. On peut imaginer que `TARGET` ne soit pas forcément linéairement dépendant de ces variables. Nous allons donc générer des combinaisons polynomiales de ces variables.


In [ ]:
from sklearn.preprocessing import PolynomialFeatures


if ENGINEER:
    # Let's keep only non null data
    ext_source = app_df[["SK_ID_CURR", "TARGET", "EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].dropna()

    # Let's create the new features
    poly = PolynomialFeatures()
    poly_feat = pd.DataFrame(poly.fit_transform(
            X=ext_source[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]],
            y=ext_source["TARGET"],
        )
    )
    poly_feat.columns=poly.get_feature_names()

    poly_feat.insert(0, "SK_ID_CURR", ext_source["SK_ID_CURR"].values)

    # Merge the new features with the original dataset
    app_df = app_df.merge(
        poly_feat,
        on="SK_ID_CURR",
        how="left",
    )

    # Draw the BoxPlots for these features
    if DRAW_PLOTS:
        vis_helpers.plot_boxes(app_df,
            plot_columns=poly_feat.columns[5:],
            categorical_column="TARGET",
        )


### Features selection

Le but ici est d'éliminer un certain nombre de variables afin d'accélérer l'entrainement et la prédiction de nos modèles. Nous souhaitons éliminer les variables qui pénaliseront le moins possible les performances de nos modèles.

Nous savons déjà que les colonnes `SK_ID_CURR` (simple identifiant sans sens métier), `1` et `x{0-2}` (variables polynomiales d'ordre 0 et 1), et `FLAG_MOBIL` (vaut toujours 1) n'apportent pas d'information. Nous allons donc les éliminer.

In [ ]:
# Let's drop the features that are not useful for the prediction
if SIMPLIFY:
    app_df = app_df.drop(
        columns=["SK_ID_CURR", "1", "x0", "x1", "x2", "FLAG_MOBIL"]
    )


Nous allons ici observer la corrélation :
- de chaque variable avec la variables cible `TARGET` : les variables les moins corrélées à `TARGET` seront a priori les moins utiles pour prédire sa valeur.
- entre les différentes variables deux à deux : si deux variables sont très corrélées, elles apportent une information redondante et nous pouvons donc en éliminer une des deux.


In [ ]:
# Let's compute the correlation matrix
if SIMPLIFY:
    app_correlations = app_df.corr().abs().sort_values(
        "TARGET", ascending=False, axis=0
    ).sort_values(
        "TARGET", ascending=False, axis=1
    )

    if DRAW_PLOTS:
        fig = px.imshow(app_correlations,
            title="Correlations between features",
            width=1200,
            height=1200,
        )
        fig.show()


Nous voyons qu'il y a des variables très peu corrélées à `TARGET` (ex. : `abs(corr("TARGET", "FLAG_EMP_PHONE")) < 0.0001`), et d'autres très corrélées entre elles (ex. : `abs(corr("LIVINGAPARTMENTS_AVG", ""LIVINGAPARTMENTS_MEDI")) > 0.99`).
Nous allons simplifier notre jeu de données en supprimant ces variables.


In [ ]:
# Let's find variables that are highly de-correlated from TARGET
if SIMPLIFY:
    corr_target_min_threshold = 0.01
    highly_decorrelated_from_target = pd.DataFrame(columns=["correlation with TARGET"])
    for col in app_correlations.columns:
        if col != "TARGET" and (
            pd.isnull(app_correlations[col]["TARGET"])
            or abs(app_correlations[col]["TARGET"]) < corr_target_min_threshold
        ):
            highly_decorrelated_from_target.loc[col] = {
                "correlation with TARGET": app_correlations[col]["TARGET"],
            }

    highly_decorrelated_from_target.sort_values(by=["correlation with TARGET"])


In [ ]:
# Let's find variables that have a highly correlated pair
if SIMPLIFY:
    corr_pair_max_threshold = 0.9
    highly_correlated = pd.DataFrame(columns=["pair", "correlation"])
    for i in range(len(app_correlations.columns)):
        for j in range(i + 1, len(app_correlations.columns)):
            if app_correlations.iloc[i, j] > corr_pair_max_threshold:
                # variables are highly correlated
                if app_correlations.iloc[0, i] > app_correlations.iloc[0, j]:
                    # first variable is more correlated with target => we want to keep it
                    keep_index = i
                    drop_index = j
                else:
                    keep_index = j
                    drop_index = i

                highly_correlated.loc[app_correlations.columns[drop_index]] = {
                    "pair": app_correlations.columns[keep_index],
                    "correlation": app_correlations.iloc[i, j],
                }

    highly_correlated.sort_values(by="correlation", ascending=False)


In [ ]:
# Drop irrelevant columns
if SIMPLIFY:
    app_df.drop(
        columns=highly_decorrelated_from_target.index,
        inplace=True,
        errors="ignore",
    )
    app_df.drop(
        columns=highly_correlated.index,
        inplace=True,
        errors="ignore",
    )

    app_df.describe(include="all")


Après simplification, nous voyons que nous avons drastiquement réduit le nombre de variables pour ne conserver que celles réellement pertinentes pour la prédiction de `TARGET`.


## Préparation des données

Nous allons transformer les données pour que nos modèles puissent les exploiter au mieux. Afin d'éviter la "fuite d'information" entre le jeu de données d'entraînement et de test, nous allons maintenant séparer notre jeu de données en deux. Les transformations seront apprises uniquement sur le jeu d'entraînement, mais appliquées aux deux jeux de données (entraînement et test).


In [ ]:
import re


# Let's rename columns to remove special characters
app_df = app_df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

# Given data
X = app_df.drop(["TARGET"], axis=1)
# Data to predict
y = app_df["TARGET"]

# Let's split the whole dataset into a training set (80% of data) and a test set (20% of data)
# The dataset will be split in a stratified way, in order to have a good distribution of the target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Save the processed data
X_train.to_csv("../data/processed/X_train.csv", index=False)
X_test.to_csv("../data/processed/X_test.csv", index=False)
y_train.to_csv("../data/processed/y_train.csv", index=False)
y_test.to_csv("../data/processed/y_test.csv", index=False)



### Normalisation des données

Afin d'éviter que certains modèles pondèrent l'importance de certaines variables à cause de leur ordre de grandeur, nous allons normaliser chaque variable afin de les ramener à une moyenne nulle et une variance de 1.


In [ ]:
# Scale each variable of the DataFrame
from sklearn.preprocessing import StandardScaler


if SCALE:
    # define scaler
    scaler = StandardScaler()

    # fit scaler on train data only, to avoid data leak
    X_train = pd.DataFrame(
        scaler.fit_transform(X_train),
        columns=X.columns,
    )
    X_test = pd.DataFrame(
        scaler.transform(X_test),
        columns=X.columns,
    )

    # Save the processed data
    X_train.to_csv("../data/processed/X_train_scaled.csv", index=False)
    X_test.to_csv("../data/processed/X_test_scaled.csv", index=False)

    X_train.describe(include="all")


Nous avons ici équilibré les ordres de grandeur de chaque variables, afin que nos futurs modèles ne soient pas influencés par leur différence.


### Imputation des valeurs manquantes

Afin d'éviter que certains modèles ne puissent être utilisés à cause des valeurs manquantes, nous allons remplacer toutes les valeurs nulles par leur meilleure estimation possible.


In [ ]:
# Impute missing values by modeling each feature with missing values as a function of other features in a round-robin fashion
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

if IMPUTE:
    if os.path.exists("../data/processed/X_train_imputed.csv") and os.path.exists("../data/processed/X_test_imputed.csv"):
        X_train = pd.read_csv("../data/processed/X_train_imputed.csv")
        X_test = pd.read_csv("../data/processed/X_test_imputed.csv")
    else:
        # define imputer
        imputer = IterativeImputer(
            n_nearest_features=min(5, int(len(X.columns) / 10)),
        )

        # fit imputer on train data only, to avoid data leak
        X_train = pd.DataFrame(
            imputer.fit_transform(X_train),
            columns=X.columns,
        )
        X_test = pd.DataFrame(
            imputer.transform(X_test),
            columns=X.columns,
        )

        # Save the processed data
        X_train.to_csv("../data/processed/X_train_imputed.csv", index=False)
        X_test.to_csv("../data/processed/X_test_imputed.csv", index=False)

    X_train.describe(include="all")


## Modélisation et évaluation des modèles

Nous allons ici entraîner différents modèles de classification binaire et évaluer leur efficacité selon trois critères :
- le **temps d'exécution** : temps nécessaire au modèle pour les phases d'apprentissage (`fit`) et de prédiction (`predict`).
- l'**interprétabilité** du modèle : la capacité du modèle à nous indiquer quelles sont les variables qui expliquent le mieux la prédiction.
- la **performance** : nous allons mesurer différentes métriques (`accuracy` = taux de prédictions positives ou négatives correctes, `precision` = taux de prédictions positives correctes, `recall` = taux de positifs correctement prédits, `F1 score` = moyenne harmonique de la `precision` et du `recall`) pour chaque modèle. Nous utiliserons le `F1 score` comme mesure principale de la performance, car nous sommes face à un problème où les deux classes de la variable prédite ne sont pas équilibrées (`TARGET=0 ~ 92%`) dans le jeu de données, et nous voulons maximiser en priorité le `recall` (bien prédire les problèmes de paiement). Nous tracerons les courbes de `Precision-Recall` et de `ROC (Receiver operating characteristic)` qui compare le `recall` au taux de faux positifs.

Afin d'améliorer la qualité du modèle et minimiser le biais de sélection du jeu d'entraînement, nous allons utiliser la méthode `StratifiedKFold` qui permet de "mixer" les résultats sur plusieurs sous-ensembles du jeu d'entraînement.

Afin de chercher les meilleurs hyper-paramètres de chacun des modèles, nous allons 

In [ ]:
# let's store the results of each model in a DataFrame
results = pd.DataFrame(columns=(
    'model',
    'params',
    'score',
    'time',
    'cv_results_',
    'best_index_',
    'confusion_matrix',
    'f1',
    'accuracy',
    'precision',
    'recall',
    'average_precision',
    'precision_recall_curve',
    'roc_auc_score',
    'roc_curve',
))


### Modèle de référence

Nous allons utiliser un modèle bête et méchant qui se contente de faire des prédictions aléatoires.


In [ ]:
## Dummy Classifier

# DummyClassifier permet de faire des prédictions aléatoires.


from sklearn.dummy import DummyClassifier


# Compute scores
best_model_score = models_helpers.find_best_params_classifier(
    X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test,
    estimator=DummyClassifier(random_state=42),
    params={
        'strategy': ['stratified', 'most_frequent', 'prior', 'uniform']
    },
)
print({
    "params": best_model_score['params'], 
    "score": best_model_score['score'],
    "predict_time": best_model_score['predict_time'],
})
results = results.append(best_model_score, ignore_index=True)


# Plot best model curves
vis_helpers.plot_classifier_results(
    classifier=best_model_score['model'],
    X_test=X_test,
    y_test=y_test,
)


### Modèle linéaire

Nous allons utiliser un modèle linéaire


In [ ]:
## Ridge Regression
# Prevents overfitting by penalizing the magnitude of coefficients with L2 regularization of the loss function

# - La norme ℓ2 du vecteur de poids peut être utilisée comme terme de régularisation de la régression linéaire.
# - Cela s'appelle la régularisation de Tykhonov, ou régression ridge.
# - La régression ridge admet toujours une solution analytique unique.
# - La régression ridge permet d'éviter le surapprentissage en restraignant l'amplitude des poids.
# - La régression ridge a un effet de sélection groupée : les variables corrélées ont le même coefficient.


from sklearn.linear_model import RidgeClassifier

if ENCODE and IMPUTE:
    # Compute scores
    best_model_score = models_helpers.find_best_params_classifier(
        X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test,
        estimator=RidgeClassifier(random_state=42),
        params={
            'alpha': np.logspace(-10, 10, 50),
            'fit_intercept': [True, False],
            'normalize': [True, False],
            'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
        },
    )
    print({
        "params": best_model_score['params'], 
        "score": best_model_score['score'],
        "predict_time": best_model_score['predict_time'],
    })
    results = results.append(best_model_score, ignore_index=True)

    # Plot best model curves
    vis_helpers.plot_classifier_results(
        classifier=best_model_score['model'],
        X_test=X_test,
        y_test=y_test,
    )


In [ ]:
if ENCODE and IMPUTE:
    top_coefficients = pd.Series(
        best_model_score[
            'model'
        ].coef_[0],
        X_train.columns,
    ).map(abs).sort_values(ascending=False).head(20)

    if DRAW_PLOTS:
        fig = px.bar(
            top_coefficients,
            color=top_coefficients.values,
            title="Top 20 variables importance",
            labels={
                "index": "Variable name",
                "value": "Coefficient",
                "color": "Coefficient",
            },
            width=1200,
            height=800,
        )
        fig.show()


### Modèle Logistique



In [ ]:
# Logistic Regression
#
# - La régression logistique modélise la probabilité qu'une observation appartienne à la classe positive comme une transformation logistique d'une combinaison linéaire des variables.
# - Les coefficients d'une régression logistique s'apprennent par maximisation de vraisemblance, mais il n'existe pas de solution explicite.
# - La vraisemblance est convexe, et de nombreux solveurs peuvent être utilisés pour trouver une solution numérique.
# - Les concepts de régularisation ℓ1 et ℓ2 s'appliquent aussi à la régression logistique.


from sklearn.linear_model import LogisticRegression


if ENCODE and IMPUTE:
    # Compute scores
    best_model_score = models_helpers.find_best_params_classifier(
        X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test,
        estimator=LogisticRegression(random_state=42),
        params={
            'penalty': ['l1', 'l2', 'elasticnet', 'none'],
            'C': np.logspace(-8, 1, 20),
            'l1_ratio': np.linspace(0.00001, 0.99999, 10),
            'solver': ['saga']
        },
    )
    print({
        "params": best_model_score['params'], 
        "score": best_model_score['score'],
        "predict_time": best_model_score['predict_time'],
    })
    results = results.append(best_model_score, ignore_index=True)

    # Plot best model curves
    vis_helpers.plot_classifier_results(
        classifier=best_model_score['model'],
        X_test=X_test,
        y_test=y_test,
    )


In [ ]:
if ENCODE and IMPUTE:
    top_coefficients = pd.Series(
        best_model_score[
            'model'
        ].coef_[0],
        X_train.columns,
    ).map(abs).sort_values(ascending=False).head(20)

    if True or DRAW_PLOTS:
        fig = px.bar(
            top_coefficients,
            color=top_coefficients.values,
            title="Top 20 variables importance",
            labels={
                "index": "Variable name",
                "value": "Coefficient",
                "color": "Coefficient",
            },
            width=1200,
            height=800,
        )
        fig.show()


In [ ]:
from sklearn.metrics import mean_squared_error

if ENCODE and IMPUTE:
    c_range = np.logspace(-8, 1, 20)
    l1_ratio = best_model_score['params']['l1_ratio']

    coefficients = pd.DataFrame(index=X_train.columns, columns=c_range)
    errors = []
    for c in c_range:
        logistic = LogisticRegression(C=c, l1_ratio=l1_ratio)
        logistic.fit(X_train, y_train)
        coefficients.loc[:, c] = logistic.coef_[0]
        errors.append(mean_squared_error(y_test, logistic.predict(X_test)))


    fig = go.Figure()
    for col in coefficients.index:
        fig.add_trace(go.Scatter(x=c_range, y=coefficients.loc[col, :], name=col,))

    fig.update_xaxes(type="log", autorange="reversed")
    fig.update_layout(
        title="Logistic regression coefficients as a function of the regularization",
        xaxis_title="log(C)",
        yaxis_title="coefficient",
        width=1200,
        height=800,
    )
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=c_range, y=errors, name="MSE"))
    fig.update_xaxes(type="log", autorange="reversed")
    fig.update_layout(
        title="Logistic regression MSE as a function of the regularization",
        xaxis_title="log(C)",
        yaxis_title="MSE",
        width=1200,
        height=800,
    )
    fig.show()


### Modèle Support Vector Machine 



In [ ]:
# Support Vector Machine

# Linear SVC
# - Les SVM (Support Vector Machines), aussi appelées en français Machines à Vecteurs de Support et parfois Séparatrices à Vaste Marge, cherchent à séparer linéairement les données.
# - La version primale résout un problème d'optimisation à p variables et est donc préférable si on a moins de variables que d'échantillons.
# - À l'inverse, la version duale résout un problème d'optimisation à n variables et est donc préférable si on a moins d'échantillons que de variables.
# - Les vecteurs de support sont les points du jeu de données qui sont les plus proches de l'hyperplan séparateur.
# - La fonction de décision peut s'exprimer uniquement en fonction du produit scalaire du point à étiqueter avec les vecteurs de support.

from sklearn.svm import SVC


if ENCODE and IMPUTE:
    # Compute scores
    best_model_score = models_helpers.find_best_params_classifier(
        X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test,
        estimator=SVC(
            cache_size=1000, 
            max_iter=1e4,
            random_state=42
        ),
        params={
            "C": np.logspace(-2, 2, 5),
            "gamma": ["scale", "auto"] + list(np.logspace(-2, 2, 5)),
            "degree": range(2, 5),
            "kernel": ["linear", "poly", "rbf"],
        },
    )
    print({
        "params": best_model_score['params'], 
        "score": best_model_score['score'],
        "predict_time": best_model_score['predict_time'],
    })
    results = results.append(best_model_score, ignore_index=True)

    # Plot best model curves
    vis_helpers.plot_classifier_results(
        classifier=best_model_score['model'],
        X_test=X_test,
        y_test=y_test,
    )


In [ ]:
if ENCODE and IMPUTE:
    # if the kernel is linear, we can use the coefficients to plot the decision boundary
    if best_model_score['params']['kernel'] == 'linear':
        top_coefficients = pd.Series(
            best_model_score['model'].coef_[0],
            X_train.columns,
        ).map(abs).sort_values(ascending=False).head(20)

        if True or DRAW_PLOTS:
            fig = px.bar(
                top_coefficients,
                color=top_coefficients.values,
                title="Top 20 variables importance",
                labels={
                    "index": "Variable name",
                    "value": "Coefficient",
                    "color": "Coefficient",
                },
                width=1200,
                height=800,
            )
            fig.show()
    else:
        print("Kernel is not linear, impossible to estimate feature importances.")


### K nearest neighbours



In [ ]:
## K-nearest-neighbours

# Linear SVC
# - Les SVM (Support Vector Machines), aussi appelées en français Machines à Vecteurs de Support et parfois Séparatrices à Vaste Marge, cherchent à séparer linéairement les données.
# - La version primale résout un problème d'optimisation à p variables et est donc préférable si on a moins de variables que d'échantillons.
# - À l'inverse, la version duale résout un problème d'optimisation à n variables et est donc préférable si on a moins d'échantillons que de variables.
# - Les vecteurs de support sont les points du jeu de données qui sont les plus proches de l'hyperplan séparateur.
# - La fonction de décision peut s'exprimer uniquement en fonction du produit scalaire du point à étiqueter avec les vecteurs de support.

from sklearn.neighbors import KNeighborsClassifier


if ENCODE and IMPUTE:
    # Compute scores
    best_model_score = models_helpers.find_best_params_classifier(
        X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test,
        estimator=KNeighborsClassifier(),
        params={
            "n_neighbors": range(2, 10),
            "weights": ["uniform", "distance"],
        },
    )
    print({
        "params": best_model_score['params'], 
        "score": best_model_score['score'],
        "predict_time": best_model_score['predict_time'],
    })
    results = results.append(best_model_score, ignore_index=True)

    # Plot best model curves
    vis_helpers.plot_classifier_results(
        classifier=best_model_score['model'],
        X_test=X_test,
        y_test=y_test,
    )


### Modèles ensemblistes

In [ ]:
## Bagging

# Linear SVC
# - Les SVM (Support Vector Machines), aussi appelées en français Machines à Vecteurs de Support et parfois Séparatrices à Vaste Marge, cherchent à séparer linéairement les données.
# - La version primale résout un problème d'optimisation à p variables et est donc préférable si on a moins de variables que d'échantillons.
# - À l'inverse, la version duale résout un problème d'optimisation à n variables et est donc préférable si on a moins d'échantillons que de variables.
# - Les vecteurs de support sont les points du jeu de données qui sont les plus proches de l'hyperplan séparateur.
# - La fonction de décision peut s'exprimer uniquement en fonction du produit scalaire du point à étiqueter avec les vecteurs de support.

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier


if ENCODE and IMPUTE:
    # Compute scores
    best_model_score = models_helpers.find_best_params_classifier(
        X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test,
        estimator=BaggingClassifier(random_state=42),
        params={
            "base_estimator": [
                DecisionTreeClassifier(),
                DecisionTreeClassifier(max_depth=2),
                DecisionTreeClassifier(max_depth=10),
                ElasticNet(),
                SVC(),
            ],
            "n_estimators": np.logspace(1, 3, 10).astype(int),
            "max_samples": np.logspace(1, 4, 10).astype(int),
            "max_features": np.linspace(5, int(len(X_train.columns)/2), 5).astype(int),
        },
    )
    print({
        "params": best_model_score['params'], 
        "score": best_model_score['score'],
        "predict_time": best_model_score['predict_time'],
    })
    results = results.append(best_model_score, ignore_index=True)

    # Plot best model curves
    vis_helpers.plot_classifier_results(
        classifier=best_model_score['model'],
        X_test=X_test,
        y_test=y_test,
    )


### Random Forest



In [ ]:
## Random Forest


from sklearn.ensemble import RandomForestClassifier


if ENCODE and IMPUTE:
    # Compute scores
    best_model_score = models_helpers.find_best_params_classifier(
        X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test,
        estimator=RandomForestClassifier(random_state=42),
        params={
            "n_estimators": np.logspace(1, 3, 10).astype(int),
            "max_depth": [None, 2, 10],
            "max_features": ["auto", "sqrt", "log2"],
        },
    )
    print({
        "params": best_model_score['params'], 
        "score": best_model_score['score'],
        "predict_time": best_model_score['predict_time'],
    })
    results = results.append(best_model_score, ignore_index=True)

    # Plot best model curves
    vis_helpers.plot_classifier_results(
        classifier=best_model_score['model'],
        X_test=X_test,
        y_test=y_test,
    )


### XGBoost



In [ ]:
## XGBoost

from xgboost import XGBClassifier


if ENCODE:
    # Compute scores
    best_model_score = models_helpers.find_best_params_classifier(
        X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test,
        estimator=XGBClassifier(random_state=42),
        params={
            "n_estimators": np.logspace(1, 3, 5).astype(int),
            "learning_rate": np.linspace(0.01, 0.5, 5),
        },
    )
    print({
        "params": best_model_score['params'], 
        "score": best_model_score['score'],
        "predict_time": best_model_score['predict_time'],
    })
    results = results.append(best_model_score, ignore_index=True)

    # Plot best model curves
    vis_helpers.plot_classifier_results(
        classifier=best_model_score['model'],
        X_test=X_test,
        y_test=y_test,
    )


### LGBMClassifier



In [ ]:
## LGBMClassifier


from lightgbm import LGBMClassifier


# Compute scores
best_model_score = models_helpers.find_best_params_classifier(
    X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test,
    estimator=LGBMClassifier(random_state=42),
    params={
        "n_estimators": np.logspace(1, 3, 5).astype(int),
        "learning_rate": np.linspace(0.01, 0.5, 5),
    },
)
print({
    "params": best_model_score['params'], 
    "score": best_model_score['score'],
    "predict_time": best_model_score['predict_time'],
})
results = results.append(best_model_score, ignore_index=True)

# Plot best model curves
vis_helpers.plot_classifier_results(
    classifier=best_model_score['model'],
    X_test=X_test,
    y_test=y_test,
)


### MLPClassifier



In [ ]:
from sklearn.neural_network import MLPClassifier


if ENCODE:
    # Compute scores
    best_model_score = models_helpers.find_best_params_classifier(
        X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test,
        estimator=MLPClassifier(random_state=42),
        params={
            "hidden_layer_sizes": [(2,), (2,3,), (2,3,5,), (2,5,3,), (3,), (3,5,), (5,3,), (5,), (10,), (20,)],
            "alpha": np.logspace(-5, 1, 5),
        },
    )
    print({
        "params": best_model_score['params'], 
        "score": best_model_score['score'],
        "predict_time": best_model_score['predict_time'],
    })
    results = results.append(best_model_score, ignore_index=True)

    # Plot best model curves
    vis_helpers.plot_classifier_results(
        classifier=best_model_score['model'],
        X_test=X_test,
        y_test=y_test,
    )


In [ ]:
## AutoML

from autosklearn.classification import AutoSklearnClassifier
from autosklearn.experimental.askl2 import AutoSklearn2Classifier
from autosklearn.metrics import roc_auc, f1



clf = AutoSklearn2Classifier(
    time_left_for_this_task=120, 
    metric=f1,
    n_jobs=4,
)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
y_pred_proba = clf.predict_proba(X_test)


# Plot best model curves
vis_helpers.plot_classifier_results(
    classifier=clf,
    X_test=X_test,
    y_test=y_test,
)


In [ ]:
cv_results = pd.DataFrame(clf.cv_results_).sort_values(by='mean_test_score', ascending=False, inplace=True)

cv_results


In [ ]:
clf.leaderboard()


In [ ]:
clf.sprint_statistics()



---

# Annexe

Les Notebooks Kaggle [Introduction: Home Credit Default Risk Competition](https://www.kaggle.com/willkoehrsen/start-here-a-gentle-introduction) (et suivants) de [Will Koehrsen](https://www.kaggle.com/willkoehrsen) ont été d'une très grande aide dans l'exploration des données.

Comment bien choisir la métrique de score d'un algorithme de classification ?

![How to Choose a Metric for Imbalanced Classification](https://machinelearningmastery.com/wp-content/uploads/2019/12/How-to-Choose-a-Metric-for-Imbalanced-Classification-latest.png)